In [108]:
import numpy as np
import numpy.linalg as lg 
from scipy.sparse.linalg import cg
from tabulate import tabulate
from time import time
from  scipy.io import mmread
A1=np.array(mmread('/content/bcsstk14.mtx.gz').todense())
# A1.shape=(1806, 1806) real symmetric positive definite
A2=np.array(mmread('/content/bcsstm19.mtx.gz').todense())
# A2.shape=(817,817) real symmetric positive definite
A3=np.array(mmread('/content/bcsstm20.mtx.gz').todense())
# A3.shape=(485,485) real symmetric positive definite
A4=np.array(mmread('/content/bcsstm24.mtx.gz').todense())
# A4.shape=(3562,3562) real symmetric positive definite
B1=np.array(mmread('/content/bcsstm21.mtx.gz').todense())
B2=np.array(mmread('/content/bcsstk16.mtx.gz').todense())
B3=np.array(mmread('/content/bcsstk17.mtx.gz').todense())
B4=np.array(mmread('/content/bcsstk18.mtx.gz').todense())
C1=np.array(mmread('/content/bcsstm22.mtx.gz').todense())
C2=np.array(mmread('/content/bcsstk27.mtx.gz').todense())
C3=np.array(mmread('/content/bcsstm26.mtx.gz').todense())

## The Conjugate Gradient Algorithm

In [109]:
def cg_iter(A, b,x0,maxiter,tol):
    num_iters = 0

    def callback(xk):
        nonlocal num_iters
        num_iters += 1
    return cg(A, b, callback=callback,tol = tol,maxiter=maxiter),num_iters

def The_Conjugate_Gradient_Algorithm(A,b,x,itermax,tol): 
    r    = p = b-np.dot(A, x)
    k    = 0   
    while lg.norm(r) > tol and k<itermax:
        alpha = np.dot(r, r)/(np.dot(np.dot(A,p), p))
        x     = x + alpha*p
        rpr   = r.copy()        
        r     = rpr - alpha*np.dot(A, p)
        beta  = (lg.norm(r))**2/lg.norm((rpr))**2
        p     = r + beta*p
        k     = k + 1   
    return x, k


# **Cluster 1:**


In [111]:
matricies=[B1,A2,A3,B2,C1,C2]
tol=1e-5
itermax = 1000

In [112]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = cg_iter(matricies[i],b,x,itermax,tol)[1] 
  t3 = time()
  y=cg(matricies[i],b,x,tol,itermax)[0]
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm  = The_Conjugate_Gradient_Algorithm(matricies[i],b,x,itermax,tol)
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']
  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+----------------+-------------+-------------+------------------+------------+-------------+
| matricies    |   condition number |   Our_nbr_iter |        time |         res |   scipy_nbr_iter |    sc_time |      sc_res |
+==============+====================+================+=============+=============+==================+============+=============+
| (3600, 3600) |       23.7154      |              3 |    66.6585  | 1.46101e-12 |                3 |   49.2222  | 2.1765e-12  |
+--------------+--------------------+----------------+-------------+-------------+------------------+------------+-------------+
| (817, 817)   |   233734           |            493 |   299.639   | 6.5024e-06  |              393 |  129.246   | 0.00022379  |
+--------------+--------------------+----------------+-------------+-------------+------------------+------------+-------------+
| (485, 485)   |   255381           |            281 |    58.8701  | 8.24194e-06 |              2

# **Cluster 2:**

In [114]:
matricies=[A1,A4,C3]
tol=1e-5
itermax = 30000

In [115]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = cg_iter(matricies[i],b,x,itermax,tol)[1] 
  t3 = time()
  y=cg(matricies[i],b,x,tol,itermax)[0]
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm  = The_Conjugate_Gradient_Algorithm(matricies[i],b,x,itermax,tol)
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']
  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+----------------+-----------+-------------+------------------+-----------+-------------+
| matricies    |   condition number |   Our_nbr_iter |      time |         res |   scipy_nbr_iter |   sc_time |      sc_res |
+==============+====================+================+===========+=============+==================+===========+=============+
| (1806, 1806) |        1.19232e+10 |          14239 |  36789.7  | 9.42032e-06 |            12822 |  16856.6  | 0.000386238 |
+--------------+--------------------+----------------+-----------+-------------+------------------+-----------+-------------+
| (3562, 3562) |        1.80764e+13 |          24352 | 396219    | 8.52663e-06 |            18229 | 154686    | 0.000538547 |
+--------------+--------------------+----------------+-----------+-------------+------------------+-----------+-------------+
| (1922, 1922) |   261567           |           2308 |   7396.21 | 9.15014e-06 |             1774 |   2780.57 | 0.0003